In [1]:
import sys
sys.path.append("/Colorization/src")  # Append path to src
import time
import numpy as np

In [2]:
from networks.dataset import load_dataset
from networks.models import get_model
from networks.losses import get_loss_func
from networks.metrics import AUC

In [3]:
root = '/Colorization/data/train2017'
annFile = '/Colorization/data/annotations/instances_train2017.json'
batch_size = 32
train_dataset = load_dataset(root, annFile, batch_size)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-xf6b2st5 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


loading annotations into memory...
Done (t=10.16s)
creating index...
index created!


In [10]:
val_root = '/Colorization/data/val2017'
val_annFile = '/Colorization/data/annotations/instances_val2017.json'
val_dataset = load_dataset(val_root, val_annFile, batch_size)

loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


In [5]:
import torch
pytorch_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
model_str = 'eccv16_pretrained'
model = get_model(model_str).to(pytorch_device)

loss_func_str = 'MSELoss'
loss_func = get_loss_func(loss_func_str, None)

optimizer = torch.optim.Adam(model.parameters(),
                             lr=3*1e-5,
                             betas=(0.9,0.99),
                             weight_decay=1e-3)

In [13]:
model.name, len(train_dataset), len(val_dataset)

('eccv16_pretrained', 3697, 157)

In [12]:
## Eval metric
auc_metric = AUC(step_size = 1.0,
                 device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

for i, (img_l, y) in enumerate(val_dataset):
    img_l = img_l.to(pytorch_device)
    y = y.to(pytorch_device)
    y_pred = model(img_l)
#     print(y_pred.shape, y.shape)
    loss = loss_func(y_pred, y)
    #loss = loss_func(model.normalize_ab(y_pred), model.normalize_ab(y))
    print(i, loss)
    
#     print(np.max(y_pred.cpu().detach().numpy()), np.min(y_pred.cpu().detach().numpy()))
#     diff = y_pred.cpu() - y.cpu()
#     print(diff.detach().numpy().shape)
#     diff = np.linalg.norm(diff.detach().numpy(), axis=1)
#     print(diff.shape)
#     diff_torch = torch.norm(y_pred-y, dim=1)
#     print(diff_torch.shape)
#     np.testing.assert_allclose(diff_torch.cpu().detach().numpy(), diff)
#     print(np.max(diff), np.min(diff))
#     print('cnt: ', np.count_nonzero(diff <= 150))
#     loss = np.sum(diff) / 2 / img_l.shape[0]
#     print(loss)
    
#     diff = y.cpu() - y.cpu()
#     print(diff.detach().numpy().shape)
#     diff = np.linalg.norm(diff.detach().numpy(), axis=1) ** 2
#     print(diff.shape)
#     print(np.max(diff), np.min(diff))
#     print('cnt: ', np.count_nonzero(diff <= 0)/ np.prod(diff.shape))
#     loss = np.sum(diff) / 2 / img_l.shape[0]
#     print(loss)
    
    auc_metric.reset()
    auc_metric.update((y_pred, y))
    
#     roc_auc_metric.reset()
#     roc_auc_metric.update((y_pred.reshape(-1), y.reshape(-1)))
#     print(roc_auc_metric.compute())

print(auc_metric.compute())

0 tensor(16218842., device='cuda:0', grad_fn=<DivBackward0>)
1 tensor(15751429., device='cuda:0', grad_fn=<DivBackward0>)
2 tensor(17927312., device='cuda:0', grad_fn=<DivBackward0>)
3 tensor(16074726., device='cuda:0', grad_fn=<DivBackward0>)
4 tensor(13535334., device='cuda:0', grad_fn=<DivBackward0>)
5 tensor(16559589., device='cuda:0', grad_fn=<DivBackward0>)
6 tensor(17736148., device='cuda:0', grad_fn=<DivBackward0>)
7 tensor(15183708., device='cuda:0', grad_fn=<DivBackward0>)
8 tensor(19484728., device='cuda:0', grad_fn=<DivBackward0>)
9 tensor(15551604., device='cuda:0', grad_fn=<DivBackward0>)
10 tensor(13447608., device='cuda:0', grad_fn=<DivBackward0>)
11 tensor(20923080., device='cuda:0', grad_fn=<DivBackward0>)
12 tensor(15453912., device='cuda:0', grad_fn=<DivBackward0>)
13 tensor(15345683., device='cuda:0', grad_fn=<DivBackward0>)
14 tensor(15757264., device='cuda:0', grad_fn=<DivBackward0>)
15 tensor(13979877., device='cuda:0', grad_fn=<DivBackward0>)
16 tensor(18236940

132 tensor(17598548., device='cuda:0', grad_fn=<DivBackward0>)
133 tensor(17305192., device='cuda:0', grad_fn=<DivBackward0>)
134 tensor(14096515., device='cuda:0', grad_fn=<DivBackward0>)
135 tensor(16980280., device='cuda:0', grad_fn=<DivBackward0>)
136 tensor(13154812., device='cuda:0', grad_fn=<DivBackward0>)
137 tensor(11580624., device='cuda:0', grad_fn=<DivBackward0>)
138 tensor(15188478., device='cuda:0', grad_fn=<DivBackward0>)
139 tensor(12395349., device='cuda:0', grad_fn=<DivBackward0>)
140 tensor(18885714., device='cuda:0', grad_fn=<DivBackward0>)
141 tensor(18441768., device='cuda:0', grad_fn=<DivBackward0>)
142 tensor(15961508., device='cuda:0', grad_fn=<DivBackward0>)
143 tensor(18568416., device='cuda:0', grad_fn=<DivBackward0>)
144 tensor(13338731., device='cuda:0', grad_fn=<DivBackward0>)
145 tensor(15082103., device='cuda:0', grad_fn=<DivBackward0>)
146 tensor(14616120., device='cuda:0', grad_fn=<DivBackward0>)
147 tensor(16775224., device='cuda:0', grad_fn=<DivBack

### Outdated

In [8]:
step = 1.0
thresholds = torch.arange(0, 150+step, step=step).to(pytorch_device)
print(type(thresholds))
print(thresholds.shape)
print(thresholds[None,None,None,...].shape, diff_torch.shape)

<class 'torch.Tensor'>
torch.Size([151])


NameError: name 'diff_torch' is not defined

In [10]:
torch.zeros_like(thresholds).shape

torch.Size([151])

In [70]:
torch.numel(thresholds)

301

In [ ]:
def AUC_metric(y_pred: torch.Tensor, y: torch.Tensor) -> float:
    step = 1
    thresholds = torch.arange(0, 150+step, step=step).to(pytorch_device)
    dist = torch.norm(y_pred-y, dim=1)
    pos_cnt = torch.sum(torch.le(dist[...,None], thresholds[None,None,None,...]), dim=(0,1,2))
    tot_cnt += torch.numel(dist)

In [23]:
dist = diff_torch
pos_cnt = torch.sum(torch.le(dist[...,None], thresholds[None,None,None,...]), dim=(0,1,2))
tot_cnt = torch.numel(dist)
print(pos_cnt, tot_cnt)
torch.sum(pos_cnt / tot_cnt) / torch.numel(pos_cnt)

tensor([     0,    466,   1834,   3982,   7190,  11262,  16062,  21437,  27440,
         33744,  40254,  47248,  54968,  62934,  71296,  79735,  88089,  96167,
        104364, 112671, 120886, 129022, 137113, 144687, 151826, 158527, 164690,
        170659, 176628, 182180, 187559, 192688, 197459, 201887, 206149, 210065,
        213915, 217441, 220753, 223982, 227035, 229954, 232607, 235138, 237417,
        239509, 241403, 243226, 244882, 246366, 247733, 248956, 249974, 250968,
        251866, 252824, 253664, 254472, 255214, 255913, 256558, 257130, 257654,
        258134, 258536, 258935, 259317, 259607, 259860, 260079, 260280, 260475,
        260645, 260810, 260948, 261070, 261195, 261323, 261437, 261531, 261607,
        261682, 261744, 261802, 261864, 261905, 261953, 261981, 262009, 262039,
        262064, 262084, 262099, 262115, 262122, 262129, 262136, 262140, 262141,
        262143, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 

tensor(0.8386, device='cuda:0')

In [8]:
from networks.dataset import load_dataset

In [9]:
root = '/Colorization/data/train2017'
annFile = '/Colorization/data/annotations/instances_train2017.json'
batch_size = 64
train_dataset = load_dataset(root, annFile, batch_size)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-lz4yu38j because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


loading annotations into memory...
Done (t=10.01s)
creating index...
index created!


In [10]:
import torch
pytorch_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
from networks.eccv16 import eccv16

model = eccv16(pretrained=True).to(pytorch_device)
    
criterion = torch.nn.MSELoss(reduction='sum')
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=3*1e-5,betas=(0.9,0.99),weight_decay=1e-3)

In [6]:
tic = time.time()
for i, (img_l, y) in enumerate(train_dataset):
    img_l = img_l.to(pytorch_device)
    y = y.to(pytorch_device)
    y_pred = model(img_l)
    print(np.linalg.norm(y_pred.detach().cpu().numpy() - y.cpu().numpy()))

    # Compute and print loss
    loss = criterion(y_pred, y)
    #loss = criterion(model.normalize_ab(y_pred), model.normalize_ab(y))
    
    toc = time.time()
    print(i, loss.item(), f'Time: {toc-tic} sec')
    tic = time.time()

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


48900.44
0 2391416832.0 Time: 2.087425708770752 sec
45685.168
1 2087234432.0 Time: 8.31327772140503 sec
50968.28
2 2597967104.0 Time: 6.674348592758179 sec
42881.805
3 1838942208.0 Time: 6.67439341545105 sec
45425.117
4 2063548032.0 Time: 6.694308042526245 sec
40564.984
5 1645593216.0 Time: 6.673346757888794 sec


KeyboardInterrupt: 